# Build the pest interface and generate the Prior

In [1]:
%matplotlib inline
import os
import shutil
import flopy
import pandas as pd
import pyemu
import helpers

In [2]:
# these are slow as...
safe_org_d = os.path.join("..", "models", "synthetic-valley-working-monthly")

#safe_org_d = os.path.join("..", "models", "synthetic-valley-working-annual")
#safe_org_d = os.path.join("..", "models", "synthetic-valley-working-advanced-annual")
assert os.path.exists(safe_org_d)

Make a copy of the safe set of model files and run mf6 in that directory

In [3]:
tmp_d = "temp"
if os.path.exists(tmp_d):
    shutil.rmtree(tmp_d)

In [4]:
sim = flopy.mf6.MFSimulation.load(sim_ws=safe_org_d)
sim.set_sim_path(tmp_d)
gwf = sim.get_model()
gwf.set_all_data_external(external_data_folder=".")
sim.write_simulation()

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package dis...
    loading package ic...
    loading package npf...
    loading package sto...
    loading package rch...
    loading package evt...
    loading package riv...
    loading package wel...
    loading package wel...
    loading package oc...
    loading package obs...
  loading solution package sv...
writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package sv...
  writing model sv...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package npf...
    writing package sto...
    writing package rch_0...
    writing package evt_0...
    writing package riv_obs...
    writing package riv-1...
    writing package pwell...
    writing package prediction...
    writing package oc...
    writing package gwf-obs...


In [5]:
sim.run_simulation()

FloPy is using the following executable to run the model: ../../../../../miniforge3/envs/tampa26mf6pp/bin/mf6
                                   MODFLOW 6
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                  VERSION 6.7.0.dev3 (preliminary) 01/14/2026
                               ***DEVELOP MODE***

        MODFLOW 6 compiled Jan 14 2026 13:48:00 with GCC version 13.4.0

This software is preliminary or provisional and is subject to 
revision. It is being provided to meet the need for timely best 
science. The software has not received final approval by the U.S. 
Geological Survey (USGS). No warranty, expressed or implied, is made 
by the USGS or the U.S. Government as to the functionality of the 
software and related material nor shall the fact of release 
constitute any such warranty. The software is provided on the 
condition that neither the USGS nor the U.S. Government shall be held 
liable for any damages resulting from the authorized or unauthorized

(True, [])

In [6]:
sim.remove_package("ims")
ims = flopy.mf6.ModflowIms(
    sim,
    print_option="summary",
    complexity="complex",
    under_relaxation=None,
    linear_acceleration="bicgstab",
    outer_maximum=500,
    inner_maximum=100,
    outer_dvclose=1e-3,
    inner_dvclose=1e-3,
)

In [7]:
sim.continue_ = True

In [8]:
sim.write_simulation()
sim.run_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package ims_-1...
  writing model sv...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package npf...
    writing package sto...
    writing package rch_0...
    writing package evt_0...
    writing package riv_obs...
    writing package riv-1...
    writing package pwell...
    writing package prediction...
    writing package oc...
    writing package gwf-obs...
FloPy is using the following executable to run the model: ../../../../../miniforge3/envs/tampa26mf6pp/bin/mf6
                                   MODFLOW 6
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                  VERSION 6.7.0.dev3 (preliminary) 01/14/2026
                               ***DEVELOP MODE***

        MODFLOW 6 compiled Jan 14 2026 13:48:00 with GCC version 13.4.0

This software is preliminary or provisional and is subject to 
revi

(True, [])

Make a `PstFrom` instance

In [9]:
pf = pyemu.utils.PstFrom(
    tmp_d,
    "model_and_pest_files",
    remove_existing=True,
    spatial_reference=gwf.modelgrid,
    zero_based=False,
    start_datetime=gwf.start_datetime,
    echo=False,
    chunk_len=1000000,
)

We are using a model-post-processing function clean up and process csv output files.  We need to tell `PstFrom` to run that function after mf6 runs:

In [10]:
helpers.process_csv_files(model_ws=pf.new_d)
pf.add_py_function("helpers.py", "process_csv_files()", is_pre_cmd=False)

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package dis...
    skipping package ic...
    skipping package npf...
    skipping package sto...
    skipping package rch...
    skipping package evt...
    skipping package riv...
    skipping package wel...
    skipping package wel...
    skipping package oc...
    skipping package obs...
    skipping package ims6...
found sv.riv.obs.csv
found sv.lake.obs.csv
35832365.690400004 80567634.3082 -44735268.6178
35832365.690400004 80567634.3082 -44735268.6178


Tell `PstFrom` to run mf6 as the "model"

In [11]:
pf.mod_sys_cmds.append("mf6")

Add the first set of model outputs as "observations" in the pest interface: "swgw-longterm-means.csv":

In [12]:
df = pd.read_csv(os.path.join(pf.new_d, "swgw-longterm-means.csv"), index_col=0)
df = pf.add_observations(
    "swgw-longterm-means.csv",
    index_cols="quantity",
    prefix="forecasts",
    obsgp="forecasts",
    ofile_sep=",",
)
print(df)

                                                                                                                        obsnme  \
oname:forecasts_otype:lst_usecol:diff-mean_quantity:lake-stage  oname:forecasts_otype:lst_usecol:diff-mean_quantity:lake-stage   
oname:forecasts_otype:lst_usecol:diff-mean_quantity:riv-swgw      oname:forecasts_otype:lst_usecol:diff-mean_quantity:riv-swgw   
oname:forecasts_otype:lst_usecol:diff-mean_quantity:wel-sum        oname:forecasts_otype:lst_usecol:diff-mean_quantity:wel-sum   
oname:forecasts_otype:lst_usecol:hist-mean_quantity:lake-stage  oname:forecasts_otype:lst_usecol:hist-mean_quantity:lake-stage   
oname:forecasts_otype:lst_usecol:hist-mean_quantity:riv-swgw      oname:forecasts_otype:lst_usecol:hist-mean_quantity:riv-swgw   
oname:forecasts_otype:lst_usecol:hist-mean_quantity:wel-sum        oname:forecasts_otype:lst_usecol:hist-mean_quantity:wel-sum   
oname:forecasts_otype:lst_usecol:pred-mean_quantity:lake-stage  oname:forecasts_otype:lst_

Now lets gather up all the output timeseries csv file we want to have as observations:

In [13]:
obs_csv_files = [
    f for f in os.listdir(pf.new_d) if f.startswith("sv.gwf") and f.endswith(".csv")
]
obs_csv_files.extend(
    [f for f in os.listdir(pf.new_d) if f.startswith("sv.lake") and f.endswith(".csv")]
)
obs_csv_files.extend(
    [f for f in os.listdir(pf.new_d) if f.startswith("sv.riv") and f.endswith(".csv")]
)
obs_csv_files.extend(
    [f for f in os.listdir(pf.new_d) if f.startswith("sv.sfr") and f.endswith(".csv")]
)
obs_csv_files.extend(
    [
        f
        for f in os.listdir(pf.new_d)
        if f.startswith("sv-budget") and f.endswith(".csv")
    ]
)
obs_csv_files

['sv.gwf.aq.csv',
 'sv.gwf.wt.csv',
 'sv.gwf.scenario.csv',
 'sv.gwf.diff.csv',
 'sv.lake.obs.csv',
 'sv.riv.obs.csv',
 'sv-budget.csv']

Loop over them and add each one to the interface

In [14]:
for obs_csv_file in obs_csv_files:
    print(obs_csv_file)
    prefix = obs_csv_file.replace(".", "-")
    df = pd.read_csv(os.path.join(pf.new_d, obs_csv_file), index_col=0)
    odf = pf.add_observations(
        obs_csv_file,
        index_cols="datetime",
        use_cols=df.columns.to_list(),
        prefix=prefix,
        ofile_sep=",",
    )
    print(odf)

sv.gwf.aq.csv
                                                                                                                                      obsnme  \
oname:sv-gwf-aq-csv_otype:lst_usecol:aq01-i2-j17_datetime:2006-01-01    oname:sv-gwf-aq-csv_otype:lst_usecol:aq01-i2-j17_datetime:2006-01-01   
oname:sv-gwf-aq-csv_otype:lst_usecol:aq01-i2-j17_datetime:2006-02-01    oname:sv-gwf-aq-csv_otype:lst_usecol:aq01-i2-j17_datetime:2006-02-01   
oname:sv-gwf-aq-csv_otype:lst_usecol:aq01-i2-j17_datetime:2006-03-01    oname:sv-gwf-aq-csv_otype:lst_usecol:aq01-i2-j17_datetime:2006-03-01   
oname:sv-gwf-aq-csv_otype:lst_usecol:aq01-i2-j17_datetime:2006-04-01    oname:sv-gwf-aq-csv_otype:lst_usecol:aq01-i2-j17_datetime:2006-04-01   
oname:sv-gwf-aq-csv_otype:lst_usecol:aq01-i2-j17_datetime:2006-05-01    oname:sv-gwf-aq-csv_otype:lst_usecol:aq01-i2-j17_datetime:2006-05-01   
...                                                                                                                       

Now some parameters.  Start with hk - the ole classic:

In [15]:
k_files = [f for f in os.listdir(pf.new_d) if f.startswith("sv.npf_k_layer")]
assert len(k_files) == gwf.dis.nlay.data

In [16]:
k_files.sort()
k_files

['sv.npf_k_layer1.txt',
 'sv.npf_k_layer2.txt',
 'sv.npf_k_layer3.txt',
 'sv.npf_k_layer4.txt',
 'sv.npf_k_layer5.txt']

We need to define some spatial correlation functions/information for the pilot points (for both interpolation from pilot points to the grid and also for the Prior covariance).  We will use a different correlation function for each property type:

In [17]:
pp_v_k = pyemu.geostats.ExpVario(contribution=1.0, a=10000)
pp_geostruct_k = pyemu.geostats.GeoStruct(variograms=pp_v_k, transform="log")
pp_v_k33 = pyemu.geostats.ExpVario(contribution=1.0, a=15000)
pp_geostruct_k33 = pyemu.geostats.GeoStruct(variograms=pp_v_k33, transform="log")
pp_v_ss = pyemu.geostats.ExpVario(contribution=1.0, a=5000)
pp_geostruct_ss = pyemu.geostats.GeoStruct(variograms=pp_v_ss, transform="log")
pp_v_sy = pyemu.geostats.ExpVario(contribution=1.0, a=2000)
pp_geostruct_sy = pyemu.geostats.GeoStruct(variograms=pp_v_sy, transform="none")

We will treat HK in layer 1 and 2 as same quantity - they will share pilot point multiplier parameters:

In [18]:
df = pf.add_parameters(
    k_files[:2],
    par_type="pilotpoints",
    pp_options={"pp_space": 3},
    lower_bound=0.1,
    upper_bound=10.0,
    geostruct=pp_geostruct_k,
    par_name_base="hk-pp-wt",
    pargp="hk-pp-wt",
)

...pypestutils detected and being used for kriging solve, trust us, you want this!
INFO:PestUtilsLib: calculated 2D kriging factors to 'hk-pp-wt_inst0pp.fac'
INFO:PestUtilsLib: all memory was freed up
...ppu_factor_filename 'model_and_pest_files/hk-pp-wt_inst0pp.fac' created, factors calculated for 1000 points


In [19]:
df.head()

,name,x,y,zone,parval1,k,i,j,parnme,tpl,...,pp_filename,pargp,dercom,offset,parchglim,parlbnd,partrans,parubnd,scale,partype
parnme,,,,,,,,,,,,,,,,,,,,,
pname:hk-pp-wt_inst:0_ptype:pp_pstyle:m_i:1_j:1_zone:1,pp_0000,750.0,19250.0,1,1.0,0,1,1,pname:hk-pp-wt_inst:0_ptype:pp_pstyle:m_i:1_j:1_zone:1,~ pname:hk-pp-wt_inst:0_ptype:pp_pstyle:m_i:1_j:1_zone:1 ~,...,model_and_pest_files/hk-pp-wt_inst0pp.dat,hk-pp-wt,1,0.0,factor,0.1,log,10.0,1.0,pilotpoints
pname:hk-pp-wt_inst:0_ptype:pp_pstyle:m_i:1_j:4_zone:1,pp_0001,2250.0,19250.0,1,1.0,0,1,4,pname:hk-pp-wt_inst:0_ptype:pp_pstyle:m_i:1_j:4_zone:1,~ pname:hk-pp-wt_inst:0_ptype:pp_pstyle:m_i:1_j:4_zone:1 ~,...,model_and_pest_files/hk-pp-wt_inst0pp.dat,hk-pp-wt,1,0.0,factor,0.1,log,10.0,1.0,pilotpoints
pname:hk-pp-wt_inst:0_ptype:pp_pstyle:m_i:1_j:7_zone:1,pp_0002,3750.0,19250.0,1,1.0,0,1,7,pname:hk-pp-wt_inst:0_ptype:pp_pstyle:m_i:1_j:7_zone:1,~ pname:hk-pp-wt_inst:0_ptype:pp_pstyle:m_i:1_j:7_zone:1 ~,...,model_and_pest_files/hk-pp-wt_inst0pp.dat,hk-pp-wt,1,0.0,factor,0.1,log,10.0,1.0,pilotpoints
pname:hk-pp-wt_inst:0_ptype:pp_pstyle:m_i:1_j:10_zone:1,pp_0003,5250.0,19250.0,1,1.0,0,1,10,pname:hk-pp-wt_inst:0_ptype:pp_pstyle:m_i:1_j:10_zone:1,~ pname:hk-pp-wt_inst:0_ptype:pp_pstyle:m_i:1_j:10_zone:1 ~,...,model_and_pest_files/hk-pp-wt_inst0pp.dat,hk-pp-wt,1,0.0,factor,0.1,log,10.0,1.0,pilotpoints
pname:hk-pp-wt_inst:0_ptype:pp_pstyle:m_i:1_j:13_zone:1,pp_0004,6750.0,19250.0,1,1.0,0,1,13,pname:hk-pp-wt_inst:0_ptype:pp_pstyle:m_i:1_j:13_zone:1,~ pname:hk-pp-wt_inst:0_ptype:pp_pstyle:m_i:1_j:13_zone:1 ~,...,model_and_pest_files/hk-pp-wt_inst0pp.dat,hk-pp-wt,1,0.0,factor,0.1,log,10.0,1.0,pilotpoints


Since the pilot points are designed to accomodate spatial heterogeneity, let's also include a layer-constant parameter to help sample a wider range of HK values:

In [20]:
df = pf.add_parameters(
    k_files[:2],
    par_type="constant",
    lower_bound=0.1,
    upper_bound=10.0,
    par_name_base="hk-cn-wt",
    pargp="hk-cn-wt",
)
df

,parnme,parval1,pargp,tpl_filename,input_filename,partype,partrans,parubnd,parlbnd,offset,dercom,parchglim,scale
pname:hk-cn-wt_inst:0_ptype:cn_pstyle:m,pname:hk-cn-wt_inst:0_ptype:cn_pstyle:m,1.0,hk-cn-wt,model_and_pest_files/hk-cn-wt_inst0_constant.csv.tpl,model_and_pest_files/mult/hk-cn-wt_inst0_constant.csv,constant,log,10.0,0.1,0.0,1,factor,1.0


Do the same for HK in layers 4 and 5 together

In [21]:
df = pf.add_parameters(
    k_files[3:],
    par_type="pilotpoints",
    pp_options={"pp_space": 3},
    lower_bound=0.1,
    upper_bound=10.0,
    geostruct=pp_geostruct_k,
    par_name_base="hk-pp-aq",
    pargp="hk-pp-aq",
)
df = pf.add_parameters(
    k_files[3:],
    par_type="constant",
    lower_bound=0.10,
    upper_bound=1.0,
    par_name_base="hk-cn-aq",
    pargp="hk-cn-aq",
)

To let us see what the actual HK array that mf6 sees, let's add that array as a set of observatitons also:

In [22]:
for k_file in [k_files[0], k_files[-1]]:
    print(k_file)
    pf.add_observations(
        k_file,
        obsgp=k_file.split(".")[1].replace("_", "-"),
        prefix=k_file.split(".")[1].replace("_", "-"),
    )

sv.npf_k_layer1.txt
sv.npf_k_layer5.txt


Setup a similar scheme of parameters to VK:

In [23]:
k33_files = [f for f in os.listdir(pf.new_d) if f.startswith("sv.npf_k33_layer")]
assert len(k_files) == gwf.dis.nlay.data
k33_files.sort()

df = pf.add_parameters(
    k33_files[:2],
    par_type="pilotpoints",
    pp_options={"pp_space": 3},
    lower_bound=0.1,
    upper_bound=10.0,
    geostruct=pp_geostruct_k33,
    par_name_base="k33-pp-wt",
    pargp="k33-pp-wt",
)
df = pf.add_parameters(
    k33_files[:2],
    par_type="constant",
    lower_bound=0.1,
    upper_bound=10.0,
    par_name_base="k33-cn-wt",
    pargp="k33-cn-wt",
)

pf.add_observations(
    k33_files[0],
    obsgp=k33_files[0].split(".")[1].replace("_", "-"),
    prefix=k33_files[0].split(".")[1].replace("_", "-"),
)

df = pf.add_parameters(
    k33_files[2],
    par_type="pilotpoints",
    pp_options={"pp_space": 3},
    lower_bound=0.01,
    upper_bound=100.0,
    geostruct=pp_geostruct_k33,
    par_name_base="k33-pp-conf",
    pargp="k33-pp-conf",
)
df = pf.add_parameters(
    k33_files[2],
    par_type="constant",
    lower_bound=0.1,
    upper_bound=10.0,
    par_name_base="k33-cn-conf",
    pargp="k33-cn-conf",
)

pf.add_observations(
    k33_files[2],
    obsgp=k33_files[2].split(".")[1].replace("_", "-"),
    prefix=k33_files[2].split(".")[1].replace("_", "-"),
)


df = pf.add_parameters(
    k33_files[3:],
    par_type="pilotpoints",
    pp_options={"pp_space": 3},
    lower_bound=0.1,
    upper_bound=10.0,
    geostruct=pp_geostruct_k33,
    par_name_base="k33-pp-aq",
    pargp="k33-pp-aq",
)
df = pf.add_parameters(
    k33_files[3:],
    par_type="constant",
    lower_bound=0.1,
    upper_bound=10.0,
    par_name_base="k33-cn-aq",
    pargp="k33-cn-aq",
)

pf.add_observations(
    k33_files[-1],
    obsgp=k33_files[-1].split(".")[1].replace("_", "-"),
    prefix=k33_files[-1].split(".")[1].replace("_", "-"),
)

...pypestutils detected and being used for kriging solve, trust us, you want this!
INFO:PestUtilsLib: calculated 2D kriging factors to 'k33-pp-wt_inst0pp.fac'
INFO:PestUtilsLib: all memory was freed up
...ppu_factor_filename 'model_and_pest_files/k33-pp-wt_inst0pp.fac' created, factors calculated for 1000 points


,obsnme,obsval,weight,obgnme
oname:npf-k33-layer5_otype:arr_i:0_j:0,oname:npf-k33-layer5_otype:arr_i:0_j:0,0.4,1.0,npf-k33-layer5
oname:npf-k33-layer5_otype:arr_i:0_j:1,oname:npf-k33-layer5_otype:arr_i:0_j:1,0.4,1.0,npf-k33-layer5
oname:npf-k33-layer5_otype:arr_i:0_j:10,oname:npf-k33-layer5_otype:arr_i:0_j:10,0.4,1.0,npf-k33-layer5
oname:npf-k33-layer5_otype:arr_i:0_j:11,oname:npf-k33-layer5_otype:arr_i:0_j:11,0.4,1.0,npf-k33-layer5
oname:npf-k33-layer5_otype:arr_i:0_j:12,oname:npf-k33-layer5_otype:arr_i:0_j:12,0.4,1.0,npf-k33-layer5
...,...,...,...,...
oname:npf-k33-layer5_otype:arr_i:9_j:5,oname:npf-k33-layer5_otype:arr_i:9_j:5,0.4,1.0,npf-k33-layer5
oname:npf-k33-layer5_otype:arr_i:9_j:6,oname:npf-k33-layer5_otype:arr_i:9_j:6,0.4,1.0,npf-k33-layer5
oname:npf-k33-layer5_otype:arr_i:9_j:7,oname:npf-k33-layer5_otype:arr_i:9_j:7,0.4,1.0,npf-k33-layer5
oname:npf-k33-layer5_otype:arr_i:9_j:8,oname:npf-k33-layer5_otype:arr_i:9_j:8,0.4,1.0,npf-k33-layer5


And SS and sy (in layer 1 only):

In [24]:
ss_files = [f for f in os.listdir(pf.new_d) if f.startswith("sv.sto_ss")]
assert len(ss_files) == gwf.dis.nlay.data

In [25]:
df = pf.add_parameters(
    ss_files[:2],
    par_type="pilotpoints",
    pp_options={"pp_space": 3},
    lower_bound=0.1,
    upper_bound=10.0,
    geostruct=pp_geostruct_ss,
    par_name_base="ss-pp-wt",
    pargp="ss-pp-wt",
)
df = pf.add_parameters(
    ss_files[:2],
    par_type="constant",
    lower_bound=0.1,
    upper_bound=10.0,
    par_name_base="ss-cn-wt",
    pargp="ss-cn-wt",
)
df = pf.add_parameters(
    ss_files[2],
    par_type="pilotpoints",
    pp_options={"pp_space": 3},
    lower_bound=0.1,
    upper_bound=10.0,
    geostruct=pp_geostruct_ss,
    par_name_base="ss-pp-conf",
    pargp="ss-pp-conf",
)
df = pf.add_parameters(
    ss_files[2],
    par_type="constant",
    lower_bound=0.1,
    upper_bound=10.0,
    par_name_base="ss-cn-conf",
    pargp="ss-cn-conf",
)
df = pf.add_parameters(
    ss_files[3:],
    par_type="pilotpoints",
    pp_options={"pp_space": 3},
    lower_bound=0.1,
    upper_bound=10.0,
    geostruct=pp_geostruct_ss,
    par_name_base="ss-pp-aq",
    pargp="ss-pp-aq",
)
df = pf.add_parameters(
    ss_files[3:],
    par_type="constant",
    lower_bound=0.1,
    upper_bound=10.0,
    par_name_base="ss-cn-aq",
    pargp="ss-cn-aq",
)

pf.add_observations(
    ss_files[0],
    obsgp=ss_files[0].split(".")[1].replace("_", "-"),
    prefix=ss_files[0].split(".")[1].replace("_", "-"),
)

pf.add_observations(
    ss_files[2],
    obsgp=ss_files[2].split(".")[1].replace("_", "-"),
    prefix=ss_files[2].split(".")[1].replace("_", "-"),
)

pf.add_observations(
    ss_files[-1],
    obsgp=ss_files[-1].split(".")[1].replace("_", "-"),
    prefix=ss_files[-1].split(".")[1].replace("_", "-"),
)

...pypestutils detected and being used for kriging solve, trust us, you want this!
INFO:PestUtilsLib: calculated 2D kriging factors to 'ss-pp-wt_inst0pp.fac'
INFO:PestUtilsLib: all memory was freed up
...ppu_factor_filename 'model_and_pest_files/ss-pp-wt_inst0pp.fac' created, factors calculated for 1000 points


,obsnme,obsval,weight,obgnme
oname:sto-ss-layer4_otype:arr_i:0_j:0,oname:sto-ss-layer4_otype:arr_i:0_j:0,0.00005,1.0,sto-ss-layer4
oname:sto-ss-layer4_otype:arr_i:0_j:1,oname:sto-ss-layer4_otype:arr_i:0_j:1,0.00005,1.0,sto-ss-layer4
oname:sto-ss-layer4_otype:arr_i:0_j:10,oname:sto-ss-layer4_otype:arr_i:0_j:10,0.00005,1.0,sto-ss-layer4
oname:sto-ss-layer4_otype:arr_i:0_j:11,oname:sto-ss-layer4_otype:arr_i:0_j:11,0.00005,1.0,sto-ss-layer4
oname:sto-ss-layer4_otype:arr_i:0_j:12,oname:sto-ss-layer4_otype:arr_i:0_j:12,0.00005,1.0,sto-ss-layer4
...,...,...,...,...
oname:sto-ss-layer4_otype:arr_i:9_j:5,oname:sto-ss-layer4_otype:arr_i:9_j:5,0.00005,1.0,sto-ss-layer4
oname:sto-ss-layer4_otype:arr_i:9_j:6,oname:sto-ss-layer4_otype:arr_i:9_j:6,0.00005,1.0,sto-ss-layer4
oname:sto-ss-layer4_otype:arr_i:9_j:7,oname:sto-ss-layer4_otype:arr_i:9_j:7,0.00005,1.0,sto-ss-layer4
oname:sto-ss-layer4_otype:arr_i:9_j:8,oname:sto-ss-layer4_otype:arr_i:9_j:8,0.00005,1.0,sto-ss-layer4


In [26]:
sy_files = [f for f in os.listdir(pf.new_d) if f.startswith("sv.sto_sy")]
assert len(sy_files) == gwf.dis.nlay.data
sy_files.sort()
sy_file = sy_files[0]
assert "layer1" in sy_file

In [27]:
df = pf.add_parameters(
    sy_file,
    par_type="pilotpoints",
    pp_options={"pp_space": 3},
    lower_bound=0.6,
    upper_bound=1.4,
    geostruct=pp_geostruct_sy,
    par_name_base="sy-pp-wt",
    pargp="sy-pp-wt",
    ult_ubound=1.0,
    transform="none",
)
df = pf.add_parameters(
    sy_file,
    par_type="constant",
    lower_bound=0.9,
    upper_bound=1.1,
    par_name_base="sy-cn-wt",
    pargp="sy-cn-wt",
    transform="none",
)
pf.add_observations(
    sy_file,
    obsgp=sy_file.split(".")[1].replace("_", "-"),
    prefix=sy_file.split(".")[1].replace("_", "-"),
)

...pypestutils detected and being used for kriging solve, trust us, you want this!
INFO:PestUtilsLib: calculated 2D kriging factors to 'sy-pp-wt_inst0pp.fac'
INFO:PestUtilsLib: all memory was freed up
...ppu_factor_filename 'model_and_pest_files/sy-pp-wt_inst0pp.fac' created, factors calculated for 1000 points


,obsnme,obsval,weight,obgnme
oname:sto-sy-layer1_otype:arr_i:0_j:0,oname:sto-sy-layer1_otype:arr_i:0_j:0,0.25,1.0,sto-sy-layer1
oname:sto-sy-layer1_otype:arr_i:0_j:1,oname:sto-sy-layer1_otype:arr_i:0_j:1,0.25,1.0,sto-sy-layer1
oname:sto-sy-layer1_otype:arr_i:0_j:10,oname:sto-sy-layer1_otype:arr_i:0_j:10,0.25,1.0,sto-sy-layer1
oname:sto-sy-layer1_otype:arr_i:0_j:11,oname:sto-sy-layer1_otype:arr_i:0_j:11,0.25,1.0,sto-sy-layer1
oname:sto-sy-layer1_otype:arr_i:0_j:12,oname:sto-sy-layer1_otype:arr_i:0_j:12,0.25,1.0,sto-sy-layer1
...,...,...,...,...
oname:sto-sy-layer1_otype:arr_i:9_j:5,oname:sto-sy-layer1_otype:arr_i:9_j:5,1.00,1.0,sto-sy-layer1
oname:sto-sy-layer1_otype:arr_i:9_j:6,oname:sto-sy-layer1_otype:arr_i:9_j:6,1.00,1.0,sto-sy-layer1
oname:sto-sy-layer1_otype:arr_i:9_j:7,oname:sto-sy-layer1_otype:arr_i:9_j:7,1.00,1.0,sto-sy-layer1
oname:sto-sy-layer1_otype:arr_i:9_j:8,oname:sto-sy-layer1_otype:arr_i:9_j:8,0.25,1.0,sto-sy-layer1


Set up some parameters for the pumping wells - we arent going to adjust these (dont we all have perfect historic water use data?!), but we will use them as decision variables later on...

In [28]:
predwel_files = [
    f
    for f in os.listdir(pf.new_d)
    if f.startswith("sv.prediction.well_stress_period_data_")
]
assert len(predwel_files) > 0
predwel_files

['sv.prediction.well_stress_period_data_223.txt',
 'sv.prediction.well_stress_period_data_237.txt',
 'sv.prediction.well_stress_period_data_196.txt',
 'sv.prediction.well_stress_period_data_182.txt',
 'sv.prediction.well_stress_period_data_169.txt',
 'sv.prediction.well_stress_period_data_155.txt',
 'sv.prediction.well_stress_period_data_141.txt',
 'sv.prediction.well_stress_period_data_140.txt',
 'sv.prediction.well_stress_period_data_154.txt',
 'sv.prediction.well_stress_period_data_168.txt',
 'sv.prediction.well_stress_period_data_183.txt',
 'sv.prediction.well_stress_period_data_197.txt',
 'sv.prediction.well_stress_period_data_236.txt',
 'sv.prediction.well_stress_period_data_222.txt',
 'sv.prediction.well_stress_period_data_208.txt',
 'sv.prediction.well_stress_period_data_234.txt',
 'sv.prediction.well_stress_period_data_220.txt',
 'sv.prediction.well_stress_period_data_181.txt',
 'sv.prediction.well_stress_period_data_195.txt',
 'sv.prediction.well_stress_period_data_142.txt',


Since the prediction well only has information for the predictive period, we can use its stress period/kper information to for other packages:

In [29]:
kper_start_pred = 1000000
for pwel_file in predwel_files:
    kper = int(pwel_file.split(".")[2].split("_")[-1]) - 1
    kper_start_pred = min(kper_start_pred, kper)
    pf.add_parameters(
        pwel_file,
        par_type="grid",
        index_cols=[0, 1, 2],
        use_cols=[3],
        upper_bound=3.0,
        lower_bound=0.0,
        pargp="predwel_kper:{0}".format(kper),
        par_name_base="predwel_kper:{0}".format(kper),
        transform="none",
        initial_value=1.0,
        par_style="m",
    )
assert kper_start_pred < 100000

Now find any remaining wel/maw files:

In [30]:
wel_files = [f for f in os.listdir(pf.new_d) if f.startswith("sv.maw_perioddata")]
ismaw = True
if len(wel_files) == 0:
    wel_files = [f for f in os.listdir(pf.new_d) if f.startswith("sv.pwell.wel_")]
    ismaw = False
assert len(wel_files) > 0
wel_files.sort()

And add parameters for them:

In [31]:
wel_files

['sv.pwell.wel_stress_period_data_10.txt',
 'sv.pwell.wel_stress_period_data_100.txt',
 'sv.pwell.wel_stress_period_data_101.txt',
 'sv.pwell.wel_stress_period_data_102.txt',
 'sv.pwell.wel_stress_period_data_103.txt',
 'sv.pwell.wel_stress_period_data_104.txt',
 'sv.pwell.wel_stress_period_data_105.txt',
 'sv.pwell.wel_stress_period_data_106.txt',
 'sv.pwell.wel_stress_period_data_107.txt',
 'sv.pwell.wel_stress_period_data_108.txt',
 'sv.pwell.wel_stress_period_data_109.txt',
 'sv.pwell.wel_stress_period_data_11.txt',
 'sv.pwell.wel_stress_period_data_110.txt',
 'sv.pwell.wel_stress_period_data_111.txt',
 'sv.pwell.wel_stress_period_data_112.txt',
 'sv.pwell.wel_stress_period_data_113.txt',
 'sv.pwell.wel_stress_period_data_114.txt',
 'sv.pwell.wel_stress_period_data_115.txt',
 'sv.pwell.wel_stress_period_data_116.txt',
 'sv.pwell.wel_stress_period_data_117.txt',
 'sv.pwell.wel_stress_period_data_118.txt',
 'sv.pwell.wel_stress_period_data_119.txt',
 'sv.pwell.wel_stress_period_data_

In [32]:
if ismaw:
    for wel_file in wel_files:
        kper = int(wel_file.split(".")[1].split("_")[-1]) - 1
        if kper < kper_start_pred:
            continue
        if kper == 0:
            continue
        print(wel_file)
        pf.add_parameters(
            wel_file,
            par_type="grid",
            par_style="m",
            par_name_base="welrate_kper:{0}".format(kper),
            pargp="welrate_kper:{0}".format(kper),
            mfile_skip=2,
            index_cols=[0],
            use_cols=[2],
            mfile_fmt="free",
            upper_bound=3.0,
            lower_bound=0.0,
            transform="none",
        )
else:
    for wel_file in wel_files:
        kper = int(wel_file.split(".")[2].split("_")[-1]) - 1
        if kper < kper_start_pred:
            continue
        pf.add_parameters(
            wel_file,
            par_type="grid",
            par_style="m",
            par_name_base="welrate_kper:{0}".format(kper),
            pargp="welrate_kper:{0}".format(kper),
            mfile_skip=2,
            index_cols=[0, 1, 2],
            use_cols=[3],
            mfile_fmt="free",
            upper_bound=3.0,
            lower_bound=0.0,
            transform="none",
        )

Add some recharge parameters for the base model - this is to try to account for the uncertainty that has been introduced through simplification...if we are using uzf, then add some small uncertainties for precip/infilt:

In [33]:
rech_files = [
    f for f in os.listdir(pf.new_d) if f.startswith("sv.rch_stress_period_data_")
]
if len(rech_files) == 0:
    uzf_pak_file = "sv.uzf_packagedata.txt"
    if os.path.exists(os.path.join(pf.new_d, uzf_pak_file)):
        uzf_files = [f for f in os.listdir(pf.new_d) if f.startswith("sv.uzf_period")]
        uzf_files.sort()
        print(uzf_files)
        assert len(uzf_files) == sim.tdis.nper.data
        for uzf_file in uzf_files:
            kper = int(uzf_file.split(".")[1].split("_")[-1]) - 1
            if kper >= kper_start_pred:  # dont add forecast parameters
                continue
            df = pf.add_parameters(
                uzf_file,
                par_type="constant",
                lower_bound=0.8,
                upper_bound=1.2,
                par_name_base="uzf-tcn_kper:{0}".format(kper),
                pargp="uzf-tcn_kper:{0}".format(kper),
                transform=None,
                index_cols=[0],
                use_cols=[1],
                mfile_skip=0,
            )
    else:
        raise Exception("didnt find any rech files or uzg package data file")

else:
    assert len(rech_files) == sim.tdis.nper.data
    rech_files.sort()
    for rech_file in rech_files:
        kper = int(rech_file.split(".")[1].split("_")[-1]) - 1
        if kper >= kper_start_pred:  # dont add forecast parameters
            continue
        df = pf.add_parameters(
            rech_file,
            par_type="constant",
            lower_bound=0.6,
            upper_bound=1.4,
            par_name_base="rech-tcn_kper:{0}".format(kper),
            pargp="rech-tcn_kper:{0}".format(kper),
            transform=None,
            index_cols=[0, 1, 2],
            use_cols=[3],
            mfile_skip=0,
        )

Now build the interface and the control file:

In [34]:
pf.build_pst(filename="pest.pst")

noptmax:0, npar_adj:1426, nnz_obs:26120


npar:1426, npar_adj:1426, nobs:26120, nnzobs:26120, filename:model_and_pest_files/pest.pst

Check the `obsval` quantities in the "* observation data" section - what are those numbers:

In [35]:
obs = pf.pst.observation_data
obs.obsval

oname:forecasts_otype:lst_usecol:diff-mean_quantity:lake-stage    9.838118e-01
oname:forecasts_otype:lst_usecol:diff-mean_quantity:riv-swgw     -3.184754e+05
oname:forecasts_otype:lst_usecol:diff-mean_quantity:wel-sum      -4.473527e+07
oname:forecasts_otype:lst_usecol:hist-mean_quantity:lake-stage    9.118163e+00
oname:forecasts_otype:lst_usecol:hist-mean_quantity:riv-swgw     -7.394037e+05
                                                                      ...     
oname:sto-sy-layer1_otype:arr_i:9_j:5                             1.000000e+00
oname:sto-sy-layer1_otype:arr_i:9_j:6                             1.000000e+00
oname:sto-sy-layer1_otype:arr_i:9_j:7                             1.000000e+00
oname:sto-sy-layer1_otype:arr_i:9_j:8                             2.500000e-01
oname:sto-sy-layer1_otype:arr_i:9_j:9                             2.500000e-01
Name: obsval, Length: 26120, dtype: float64

So if the `obsval` values are all the existing model output values, then if we run the model again just the same way, we should have a phi of zero - a great check!. Let's do that:

In [36]:
pst = pf.pst
pst.control_data.noptmax = 0
pst.write(os.path.join(pf.new_d, "pest.pst"), version=2)

noptmax:0, npar_adj:1426, nnz_obs:26120


In [37]:
pyemu.os_utils.run("pestpp-ies pest.pst", cwd=pf.new_d)

pestpp-ies pest.pst


             pestpp-ies: a GLM iterative ensemble smoother

                   by the PEST++ development team

...processing command line: ' pestpp-ies pest.pst'
...using serial run manager


version: 5.2.25
binary compiled on Jan  6 2026 at 12:27:13
using control file: "pest.pst"
in directory: "/Users/jwhite/Dev/modflow-pest-tampa-2026/exercises/notebooks/model_and_pest_files"
on host: "macminiller.local"
on a(n) apple operating system
with release configuration
started at 01/14/26 08:54:04

processing control file pest.pst

Note: 'NOPTMAX' == 0, switching to forgiveness mode when checking inputs

noptmax = 0, resetting max_run_fail = 1
parameter warning: 1 parameters are at upper bound
checking model IO files...done
              starting serial run manager ...


  ---  initializing  ---  

  ---  'noptmax'=0, running control file parameter values and quitting  ---  
...saving control file parameter ensemble to  pest.base.par.csv
...initializing prior parameter 

In [38]:
pst.set_res(os.path.join(pf.new_d, "pest.base.rei"))
pst.phi

np.float64(8.07119289772424e-09)

assert pst.phi < 1.e7

now we need to "fix" the pumping well pars - those are for later when we do optimization

In [39]:
par = pst.parameter_data

wellpars = par.loc[par.parnme.str.contains("wel"), :]
assert len(wellpars) > 0
par.loc[wellpars.parnme, "partrans"] = "fixed"

Now generate a Prior parameter ensemble (which the parameter bound and geostat info we passed to `PstFrom` above)

In [40]:
pe = pf.draw(num_reals=1000)

building diagonal cov
processing  name:struct1,nugget:0.0,structures:
name:var1,contribution:1.0,a:10000.0,anisotropy:1.0,bearing:0.0

working on pargroups ['hk-pp-wt']
build cov matrix
done
getting diag var cov 104
scaling full cov by diag var cov
working on pargroups ['hk-pp-aq']
build cov matrix
done
getting diag var cov 104
scaling full cov by diag var cov
processing  name:struct1,nugget:0.0,structures:
name:var1,contribution:1.0,a:15000.0,anisotropy:1.0,bearing:0.0

working on pargroups ['k33-pp-wt']
build cov matrix
done
getting diag var cov 104
scaling full cov by diag var cov
working on pargroups ['k33-pp-conf']
build cov matrix
done
getting diag var cov 104
scaling full cov by diag var cov
working on pargroups ['k33-pp-aq']
build cov matrix
done
getting diag var cov 104
scaling full cov by diag var cov
processing  name:struct1,nugget:0.0,structures:
name:var1,contribution:1.0,a:5000.0,anisotropy:1.0,bearing:0.0

working on pargroups ['ss-pp-wt']
build cov matrix
done
getting d

We need to enforce parameter bounds on those realizations, save it and add an arg to the control file to tell ies to use it:

In [41]:
pe.enforce()
pe.to_csv(os.path.join(pf.new_d, "prior.csv"))
pst.pestpp_options["ies_par_en"] = "prior.csv"
pst.pestpp_options["ies_num_reals"] = 30
pst.control_data.noptmax = -1
pst.write(os.path.join(pf.new_d, "pest.pst"), version=2)

noptmax:-1, npar_adj:1066, nnz_obs:26120


Run the first realization in the ensemble:

In [42]:
pst.parameter_data.loc[pe.columns, "parval1"] = pe.iloc[0, :].values
pst.control_data.noptmax = 0
pst.write(os.path.join(pf.new_d, "test.pst"), version=2)
pyemu.os_utils.run("pestpp-ies test.pst", cwd=pf.new_d)

noptmax:0, npar_adj:1066, nnz_obs:26120
pestpp-ies test.pst


             pestpp-ies: a GLM iterative ensemble smoother

                   by the PEST++ development team

...processing command line: ' pestpp-ies test.pst'
...using serial run manager


version: 5.2.25
binary compiled on Jan  6 2026 at 12:27:13
using control file: "test.pst"
in directory: "/Users/jwhite/Dev/modflow-pest-tampa-2026/exercises/notebooks/model_and_pest_files"
on host: "macminiller.local"
on a(n) apple operating system
with release configuration
started at 01/14/26 08:54:15

processing control file test.pst

Note: 'NOPTMAX' == 0, switching to forgiveness mode when checking inputs

noptmax = 0, resetting max_run_fail = 1
parameter warning: 2 parameters are at lower bound
parameter warning: 1 parameters are at upper bound
checking model IO files...done
              starting serial run manager ...


  ---  initializing  ---  

  ---  'noptmax'=0, running control file parameter values and quitting  ---  
...sa

Extra:  run a small prior monte carlo:

In [43]:
# pyemu.os_utils.start_workers(
#     pf.new_d,
#     "pestpp-ies",
#     "pest.pst",
#     worker_root=".",
#     num_workers=10,
#     master_dir="master_prior_mc",
# )